<a href="https://colab.research.google.com/github/VoidEnd315/CNN/blob/main/local%20dataset%20training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!tar -xzvf "/content/trainingSet.tar.gz" -C "/content"

In [ ]:
!tar -xzvf "/content/testSet.tar.gz" -C "/content"

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Function to load and preprocess images
def load_mnist_from_folder(folder_path):
    images = []
    labels = []
    for digit_folder in os.listdir(folder_path):
        digit_path = os.path.join(folder_path, digit_folder)
        if os.path.isdir(digit_path):
            for img_file in os.listdir(digit_path):
                img_path = os.path.join(digit_path, img_file)
                img = load_img(img_path, color_mode='grayscale', target_size=(28, 28))
                img_array = img_to_array(img)
                images.append(img_array)
                labels.append(int(digit_folder))
    return np.array(images), np.array(labels)

# Load and preprocess the data
data_folder = '/content/trainingSet'  # Replace with your actual path
X, y = load_mnist_from_folder(data_folder)

# Normalize pixel values
X = X.astype('float32') / 255.0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

# Save the model
model.save('mnist_cnn_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
840/840 ━━━━━━━━━━━━━━━━━━━━ 30s 32ms/step - accuracy: 0.8222 - loss: 0.5844 - val_accuracy: 0.9682 - val_loss: 0.1041
Epoch 2/10
840/840 ━━━━━━━━━━━━━━━━━━━━ 26s 31ms/step - accuracy: 0.9770 - loss: 0.0744 - val_accuracy: 0.9763 - val_loss: 0.0776
Epoch 3/10
840/840 ━━━━━━━━━━━━━━━━━━━━ 26s 31ms/step - accuracy: 0.9837 - loss: 0.0505 - val_accuracy: 0.9778 - val_loss: 0.0701
Epoch 4/10
840/840 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.9901 - loss: 0.0327 - val_accuracy: 0.9808 - val_loss: 0.0722
Epoch 5/10
840/840 ━━━━━━━━━━━━━━━━━━━━ 26s 31ms/step - accuracy: 0.9913 - loss: 0.0265 - val_accuracy: 0.9827 - val_loss: 0.0683
Epoch 6/10
840/840 ━━━━━━━━━━━━━━━━━━━━ 31s 36ms/step - accuracy: 0.9928 - loss: 0.0214 - val_accuracy: 0.9874 - val_loss: 0.0530
Epoch 7/10
840/840 ━━━━━━━━━━━━━━━━━━━━ 46s 43ms/step - accuracy: 0.9943 - loss: 0.0162 - val_accuracy: 0.9885 - val_loss: 0.0541
Epoch 8/10
840/840 ━━━━━━━━━━━━━━━━━━━━ 38s 39ms/step - accuracy: 0.9947 - loss: 0.0143 - 


Test accuracy: 0.9839285612106323


In [4]:
def predict_digit(model, image_path):
    # Load and preprocess the image
    img = load_img(image_path, color_mode='grayscale', target_size=(28, 28))
    img_array = img_to_array(img)
    img_array = img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make prediction
    prediction = model.predict(img_array)
    digit = np.argmax(prediction)
    confidence = np.max(prediction)

    return digit, confidence

# Example usage of the prediction function
if __name__ == "__main__":
    # Load the saved model
    loaded_model = tf.keras.models.load_model('mnist_cnn_model.h5')

    # Replace with the path to your image file
    image_path = '/content/testSet/img_1.jpg'

    predicted_digit, confidence = predict_digit(loaded_model, image_path)
    print(f"Predicted digit: {predicted_digit}")
    print(f"Confidence: {confidence:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Predicted digit: 2
Confidence: 1.00
